In [1]:
import pandas as pd
import spacy
from tqdm import tqdm
from spacy.matcher import PhraseMatcher

In [2]:
csv_file = 'preprocessed_0f_haiti.csv' 

In [3]:
df = pd.read_csv(csv_file)

In [5]:
df.head()

,text,date,original,genre,related,PII,request,offer,aid_related,medical_help,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,actionable_haiti,processed_text
0,UN reports Leogane 80-90 destroyed. Only Hospi...,17-01-2010,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,un reports leogane destroyed only hospital st ...
1,We are at Gressier we needs assistance right a...,17-01-2010,Se gressier nou an difikilte tanpri vin ede nou,direct,1,0,1,0,1,1,...,1,1,0,0,0,0,0,1,0,we are at gressier we needs assistance right a...
2,"Delmas 33 in Silo, need water.",17-01-2010,Delma 33 silo gen problem dlo,direct,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,delmas in silo need water
3,"SOS SOS, please provide police officers on the...",17-01-2010,EMGENCY EMGENCY SI POLIS LA TE KA BAY PREZANS ...,direct,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,sos sos please provide police officers on the ...
4,Is the Hurricane over or is it not over,17-01-2010,Cyclone nan fini osinon li pa fini,direct,1,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,is the hurricane over or is it not over


In [6]:
def extract_locations(text):
    nlp = spacy.load("en_core_web_sm")
    matcher = PhraseMatcher(nlp.vocab)
    # Add custom location names to the matcher
    location_patterns = []
    for loc in location_patterns:
        matcher.add("LOCATION", None, nlp(loc))

    doc = nlp(text)
    locations = []
    
    # Extract locations using NER
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'LOC':
            locations.append(ent.text)

    # Extract additional locations using the custom matcher
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        locations.append(span.text)
    
    return locations

In [7]:
all_locations1 = []
flags=[]
for text in tqdm(df['processed_text'], desc="Extracting Locations", total=len(df)):
    locations = extract_locations(text)
    all_locations1.extend(locations)
    
    # Set the flag to 1 if locations were found, otherwise 0
    if locations:
        flags.append(1)
    else:
        flags.append(0)

# Add the flag column to the DataFrame
df['flag1'] = flags
# Print or use the list of all locations
print(all_locations1)

Extracting Locations:   0%|          | 0/5182 [00:00<?, ?it/s]C:\Users\deepp\AppData\Roaming\Python\Python312\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\deepp\AppData\Roaming\Python\Python312\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Extracting Locations:   1%|          | 42/5182 [00:54<1:50:32,  1.29s/it]


KeyboardInterrupt: 

In [8]:
count_ones = df['flag1'].sum()

In [9]:
count_ones

446

In [7]:
locations_df = pd.DataFrame(all_locations1, columns=['Location'])

In [13]:
locations_df

,Location
0,japan
1,أعلنت
2,japan
3,japan
4,japan
...,...
758,west coast
759,japan
760,west coast
761,japan


In [14]:
locations_df.to_csv('extracted_locations_japan.csv', index=False)

In [15]:
from geopy.geocoders import Nominatim

In [16]:
def get_coordinates(location):
    geolocator = Nominatim(user_agent="location_finder")
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.latitude, location_info.longitude
        else:
            return None, None
    except:
        return None, None

In [17]:
get_coordinates('raichur')

(16.083333, 77.166667)

In [18]:
locations_data = []

In [19]:
unique_locations = set(all_locations)
for location in tqdm(unique_locations, desc="Geocoding Locations"):
    frequency = all_locations.count(location)
    latitude, longitude = get_coordinates(location)
    if latitude is not None and longitude is not None:
        locations_data.append({'Location': location, 'Frequency': frequency, 'Latitude': latitude, 'Longitude': longitude})


Geocoding Locations: 100%|█████████████████████████████████████████████████████████████| 80/80 [01:43<00:00,  1.29s/it]


In [20]:
locations_coordinate = pd.DataFrame(locations_data)

In [21]:
locations_coordinate

,Location,Frequency,Latitude,Longitude
0,rápida,1,-3.467559,-80.233213
1,mahadev,1,24.622499,82.173876
2,delhi,1,28.627393,77.171695
3,the united states,1,47.828163,-122.598365
4,united states turkiye,1,40.759698,-73.920210
...,...,...,...,...
62,china,9,35.000074,104.999927
63,الأولى,1,31.180487,31.200973
64,اپنی,1,43.084762,40.817049
65,southafrica,1,-26.169444,28.194085


In [22]:
locations_coordinate.to_csv('coordinate_japan.csv', index=False)